In [4]:
import numpy as np
import pandas as pd
import re
import spacy
import warnings
from nltk.corpus import stopwords
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud ,STOPWORDS
stopword=set(STOPWORDS)

In [5]:
train = pd.read_csv("C:/Users/setuc/Insofe internship/train.csv")
test = pd.read_csv("c:/users/setuc/Insofe internship/test.csv")

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
test.tail(10)

,id,comment_text
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...
153156,fffb5451268fb5ba,""" \n\n == Unicorn lair discovery == \n\n Suppo..."
153157,fffc2b34bbe61c8d,:Disagree. Soviet railways need their own arti...
153158,fffc489742ffe69b,This idiot can't even use proper grammar when ...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."
153163,ffffce3fb183ee80,""" \n :::Stop already. Your bullshit is not wel..."


In [8]:
nrow_train=train.shape[0]
nrow_test=test.shape[0]
sum=nrow_train+nrow_test
print("       : train : test")
print("rows   :",nrow_train,":",nrow_test)
print("perc   :",round(nrow_train*100/sum),"   :",round(nrow_test*100/sum))

       : train : test
rows   : 159571 : 153164
perc   : 51    : 49


In [9]:
train.toxic.value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [10]:
pd.crosstab(train.toxic, train.severe_toxic)

severe_toxic,0,1
toxic,,
0,144277,0
1,13699,1595


# As we might expect, all severe toxic comments are also toxic comments, but not all toxic comments are severe

In [11]:
pd.crosstab(train.toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

obscene             0                                    1                     \
threat              0                    1               0                  1   
insult              0          1         0      1        0         1        0   
identity_hate       0    1     0    1    0  1   0  1     0   1     0    1   0   
toxic                                                                           
0              143346   54   301   28   22  0   3  0   317   3   181   18   2   
1                5707  139  1229  141  124  8  17  3  1916  41  4789  883  15   

obscene                 
threat                  
insult           1      
identity_hate    0   1  
toxic                   
0                2   0  
1              195  87

In [12]:
pd.crosstab(train.severe_toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

obscene             0                                    1                     \
threat              0                    1               0                  1   
insult              0          1         0      1        0         1        0   
identity_hate       0    1     0    1    0  1   0  1     0   1     0    1   0   
severe_toxic                                                                    
0              149012  190  1516  162  135  7  19  3  2075  38  3981  636  13   
1                  41    3    14    7   11  1   1  0   158   6   989  265   4   

obscene                 
threat                  
insult           1      
identity_hate    0   1  
severe_toxic            
0              133  56  
1               64  31

In [13]:
pd.crosstab(train.severe_toxic, [train.obscene, train.threat, train.insult, train.identity_hate])

obscene             0                                    1                     \
threat              0                    1               0                  1   
insult              0          1         0      1        0         1        0   
identity_hate       0    1     0    1    0  1   0  1     0   1     0    1   0   
severe_toxic                                                                    
0              149012  190  1516  162  135  7  19  3  2075  38  3981  636  13   
1                  41    3    14    7   11  1   1  0   158   6   989  265   4   

obscene                 
threat                  
insult           1      
identity_hate    0   1  
severe_toxic            
0              133  56  
1               64  31

In [14]:
train.head(50)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [15]:
print(train.comment_text[2])

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.


In [16]:
test.head(10)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [17]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [18]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test)))
print('Total rows in train is {}'.format(len(train)))
print(train[cols_target].sum())

Total rows in test is 153164
Total rows in train is 159571
obscene           8449
insult            7877
toxic            15294
severe_toxic      1595
identity_hate     1405
threat             478
dtype: int64


# Let's look at the character length for the rows in the training data & test data and record these

In [19]:
train['char_length'] = train['comment_text'].apply(lambda x: len(str(x)))

In [20]:
test['char_length'] = test['comment_text'].apply(lambda x: len(str(x)))

In [21]:
def quora_cleaner(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    
    return text

In [22]:
# clean the comment_text in train_df
cleaned_train_comment = []
for i in range(0,len(train)):
    cleaned_comment = quora_cleaner(train['comment_text'][i])
    cleaned_train_comment.append(cleaned_comment)
train['comment_text'] = pd.Series(cleaned_train_comment).astype(str)

In [23]:
# clean the comment_text in test_df
cleaned_test_comment = []
for i in range(0,len(test)):
    cleaned_comment = quora_cleaner(test['comment_text'][i])
    cleaned_test_comment.append(cleaned_comment)
test['comment_text'] = pd.Series(cleaned_test_comment).astype(str)

In [24]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,char_length
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,264
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0,112
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0,233
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0,622
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,67


# Define X from entire train & test data for use in tokenization by Vectorizer

In [25]:
train = train.drop('char_length',axis=1)

In [26]:
X = train.comment_text
test_X = test.comment_text

In [27]:
print(X.shape, test_X.shape)

(159571,) (153164,)


In [28]:
print(len(X))

159571


In [29]:
# taking comment from train and test and making a single dataframe
train_ = train['comment_text']
test_ = test['comment_text']

alldata = pd.concat([train_, test_], axis=0)

alldata = pd.DataFrame(alldata)

# Vectorize the data

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
# define a CountVectorizer
countvec = CountVectorizer(max_features = 1500, ngram_range=(1, 2))

In [32]:
alldata['comment_text'] = alldata['comment_text'].map(lambda x: quora_cleaner(x))

In [33]:
# transform the text data using CountVectorizer
countvecdata = countvec.fit_transform(alldata['comment_text'])

In [34]:
# convert the data to a matrix
countvec_df = pd.DataFrame(countvecdata.todense()) 

In [35]:
# adding column headers
countvec_df.columns = ['col' + str(x) for x in countvec_df.columns]

In [36]:
# slice the data to train and test
countvec_df_train = countvecdata[:len(train_)] 
countvec_df_test = countvecdata[len(train_):]

In [37]:
# converting to float32
countvec_df_train_ = countvec_df_train.astype('float32')
countvec_df_test_ = countvec_df_test.astype('float32')

In [38]:
import lightgbm as lgbm
import re

In [39]:
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

preds = np.zeros((test.shape[0], len(col)))

In [40]:
# parameters for LightGBMClassifier
params = {
    'objective' :'binary',
    'learning_rate' : 0.02,
    'num_leaves' : 76,
    'feature_fraction': 0.64, 
    'bagging_fraction': 0.8, 
    'bagging_freq':1,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}

In [41]:
# for making train - valid sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [42]:
# making prediction for each column

for i, j in enumerate(col):
    print('fitting column : '+j)
    # making test - valid sets
    X_train, X_valid, Y_train, Y_valid = train_test_split(countvec_df_train_,  train[j], random_state=7, test_size=0.33)
    
    # making lgbm datasets for train and valid
    d_train = lgbm.Dataset(X_train, Y_train)
    d_valid = lgbm.Dataset(X_valid, Y_valid)
    
    # training with early stop
    bst = lgbm.train(params, d_train, 500, valid_sets=[d_valid], verbose_eval=100)
    
    # making prediciton for one column
    print('predicting for :' +j)
    preds[:,i] = bst.predict(countvec_df_test_)

print('Fininshed Training')

fitting column : toxic
[100]	valid_0's binary_logloss: 0.204397
[200]	valid_0's binary_logloss: 0.15826
[300]	valid_0's binary_logloss: 0.145402
[400]	valid_0's binary_logloss: 0.140186
[500]	valid_0's binary_logloss: 0.137676
predicting for :toxic
fitting column : severe_toxic
[100]	valid_0's binary_logloss: 0.0910319
[200]	valid_0's binary_logloss: 0.0346623
[300]	valid_0's binary_logloss: 0.0279061
[400]	valid_0's binary_logloss: 0.0272453
[500]	valid_0's binary_logloss: 0.0275936
predicting for :severe_toxic
fitting column : obscene
[100]	valid_0's binary_logloss: 0.129492
[200]	valid_0's binary_logloss: 0.0784343
[300]	valid_0's binary_logloss: 0.0708614
[400]	valid_0's binary_logloss: 0.0688122
[500]	valid_0's binary_logloss: 0.0681472
predicting for :obscene
fitting column : threat
[100]	valid_0's binary_logloss: 0.0767213
[200]	valid_0's binary_logloss: 0.0179674
[300]	valid_0's binary_logloss: 0.0107236
[400]	valid_0's binary_logloss: 0.00986247
[500]	valid_0's binary_logloss:

In [45]:
print('predicting for :' +j)
preds[:,i] = bst.predict(test(comment_text))

predicting for :identity_hate


NameError: name 'comment_text' is not defined